In [6]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance
from yellowbrick.contrib.scatter import ScatterVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score, adjusted_rand_score
from matplotlib.ticker import FixedLocator, FixedFormatter
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics.cluster import adjusted_rand_score

In [7]:
data = pd.read_csv('C:\\Users\\dtulomba\\Documents\\ProjetsCours\\projet5\\olist_customers_dataset.csv\\data_cleaned.csv', sep=',')

In [8]:
data = data.rename(columns={'recency': 'Nb_inactive_days'})

In [9]:
data = data.rename(columns={'monetary': 'montant'})

In [10]:
data

,customer_unique_id,Nb_inactive_days,frequency,montant,note_com,frais_livraison,temps_moyen_livraison,Alimentation et Boissons,Auto et Moto,Autres,Livres et Médias,Maison et Jardin,Mode,Santé et Beauté,Sports et Loisirs,Électronique
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0,0,0,0,0,1,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0,0,0,0,0,0,0,1,0,0
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0,0,0,0,0,0,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0,0,0,0,0,0,0,0,0,1
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92488,fffcf5a5ff07b0908bd4e2dbc735a684,446,2,4134.84,5.0,248.71,27.0,0,0,0,0,0,0,2,0,0
92489,fffea47cd6d3cc0a88bd621562a9d061,261,1,84.58,4.0,19.69,30.0,0,0,0,0,0,0,1,0,0
92490,ffff371b4d645b6ecea244b27531430a,567,1,112.46,5.0,22.56,14.0,0,1,0,0,0,0,0,0,0
92491,ffff5962728ec6157033ef9805bacc48,118,1,133.69,5.0,18.69,11.0,0,0,0,0,0,1,0,0,0


In [11]:
#segment_1 = data[['customer_unique_id','Nb_inactive_days','frequency','montant']]  # Correspond à table RFM
segment_2 = data[['customer_unique_id','Nb_inactive_days','frequency','montant','note_com','frais_livraison','temps_moyen_livraison']]
#segment_3 = data

segment_1.head().rename(columns={'Nb_inactive_days':'recency'})

In [13]:
segment_2.head().rename(columns={'Nb_inactive_days':'recency'})

,customer_unique_id,recency,frequency,montant,note_com,frais_livraison,temps_moyen_livraison
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90,5.0,12.00,6.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19,4.0,8.29,3.0
2,0000f46a3911fa3c0805444483337064,536,1,86.22,3.0,17.22,25.0
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62,4.0,17.63,20.0
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89,5.0,16.89,13.0


### Explication de l'algorithme DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) est un algorithme de clustering non paramétrique qui peut identifier des groupes de points denses dans un ensemble de données. L'un des avantages de DBSCAN est qu'il peut identifier des clusters de formes arbitraires et peut être efficace pour les ensembles de données contenant du bruit ou des valeurs aberrantes.

L'algorithme DBSCAN fonctionne en définissant deux paramètres clés : epsilon (ε) et minPts. Epsilon est la distance maximale entre deux points pour qu'ils soient considérés comme faisant partie du même cluster, tandis que minPts est le nombre minimum de points requis pour former un cluster. Ces deux paramètres définissent ensemble la densité requise pour qu'un groupe de points soit considéré comme un cluster.

L'algorithme DBSCAN fonctionne comme suit :

    - Sélectionnez un point au hasard qui n'a pas encore été visité.

    - Trouvez tous les points dans un rayon epsilon autour du point sélectionné.

    - Si le nombre de points dans ce rayon est supérieur ou égal à minPts, créez un nouveau cluster et ajoutez tous les   points trouvés dans ce rayon.

    - Répétez les étapes 2 et 3 pour tous les points dans le nouveau cluster.

    - Si le nombre de points dans le rayon est inférieur à minPts, le point est marqué comme du bruit et n'appartient à aucun cluster.

    - Répétez les étapes 1 à 5 jusqu'à ce que tous les points aient été visités.

À la fin de l'algorithme, certains points peuvent rester marqués comme du bruit, ce qui signifie qu'ils ne sont pas suffisamment proches d'autres points pour former un cluster.

L'un des avantages de DBSCAN est qu'il peut identifier des clusters de formes arbitraires, contrairement à d'autres algorithmes de clustering tels que k-means qui supposent que les clusters sont sphériques. De plus, DBSCAN peut gérer des ensembles de données contenant du bruit ou des valeurs aberrantes car il ne les inclura pas dans un cluster à moins qu'ils ne répondent aux critères de densité définis par les paramètres epsilon et minPts.

Cependant, il peut être difficile de choisir les bons paramètres pour DBSCAN, car ils dépendent de la densité de l'ensemble de données. Si epsilon est trop grand, des clusters différents peuvent être fusionnés ensemble, tandis que si epsilon est trop petit, des clusters différents peuvent être considérés comme séparés. De même, si minPts est trop petit, de petits groupes de points peuvent être considérés comme des clusters, tandis que si minPts est trop grand, certains points peuvent être considérés comme du bruit même s'ils font partie d'un cluster.

En résumé, DBSCAN est un algorithme de clustering non paramétrique efficace pour l'identification de clusters de formes arbitraires dans les ensembles de données denses. Cependant, le choix des paramètres peut être difficile et dépend de la densité de l'ensemble de données.


##### DBSCAN pour ma table segment_2

In [22]:
# StandardScaler
scaler = StandardScaler()

segment_2_copy=segment_2.drop(['customer_unique_id'], axis=1).copy()
segment_2_sample_x = segment_2_copy.sample(frac = .30, random_state = 42) # faire le cluster avec l'ensemble des données
segment_2_scaled = scaler.fit_transform (segment_2_sample_x)

Finding Best hyperparameters for DBSCAN using Silhouette Coefficient

In [28]:
%%time
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Initialiser une liste pour stocker les scores de silhouette pour chaque nombre de clusters
silhouette_scores = []

# Itérer sur différents nombres de clusters pour le clustering agglomératif
for n_clusters in range(3, 7):
    # Créer un objet AgglomerativeClustering pour le clustering
    clustering = AgglomerativeClustering(n_clusters=n_clusters)
    # Exécuter le clustering sur les données
    clustering_labels = clustering.fit_predict(segment_2_scaled)
    # Calculer le score de silhouette pour les clusters
    silhouette_avg = silhouette_score(segment_2_scaled, clustering_labels)
    # Ajouter le score de silhouette à la liste
    silhouette_scores.append(silhouette_avg)

# Tracer le nombre de clusters en fonction du score de silhouette
plt.plot(range(3, 7), silhouette_scores)
plt.xlabel('Nombre de clusters')
plt.ylabel('Score de silhouette')
plt.show()

Wall time: 6min 17s


Paramètres pour le dbscan

Ici on essaye de definir les hyperparamètres le plus adaptés 

In [24]:
silhouette_scores

,eps,min sample,nb cluster,silouhette score,n noise
7,1.2,25,4,0.137127,867
6,1.1,25,4,0.135392,1020
3,0.8,25,3,0.117826,2227
5,1.0,25,5,0.115156,1272
4,0.9,25,6,0.105448,1647
2,0.7,25,7,0.049069,2845
0,0.5,25,8,0.031685,4967
1,0.6,25,9,0.030140,3671


# calinski_harabasz_score en fonction du nombre de cluster

Clustering en utilisant les meilleurs hyperparamètres

In [25]:
%%time
from sklearn.metrics import calinski_harabasz_score
# Initialiser une liste pour stocker les scores Calinski-Harabasz pour chaque nombre de clusters
ch_scores = []

# Itérer sur différents nombres de clusters pour le clustering agglomératif
for n_clusters in range(3, 7):
    # Créer un objet AgglomerativeClustering pour le clustering
    clustering = AgglomerativeClustering(n_clusters=n_clusters)
    # Exécuter le clustering sur les données
    clustering_labels = clustering.fit_predict(segment_2_scaled)
    # Calculer le score Calinski-Harabasz pour les clusters
    ch_score = calinski_harabasz_score(segment_2_scaled, clustering_labels)
    # Ajouter le score Calinski-Harabasz à la liste
    ch_scores.append(ch_score)

# Tracer le nombre de clusters en fonction du score Calinski-Harabasz
plt.plot(range(3, 7), ch_scores)
plt.xlabel('Nombre de clusters')
plt.ylabel('Score Calinski-Harabasz')
plt.show()

Wall time: 20.5 s


0.13712708262394666

# Dendogramme

In [26]:
%%time
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
model = model.fit(segment_2_scaled)

plt.figure(figsize=(17, 6))
plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

Silhouette Coefficient: 0.137


Choix du nombre de cluster et caractérisation

In [27]:
for nb_clusters in range(3, 7):
    model = AgglomerativeClustering(n_clusters=nb_clusters).fit(segment_2_scaled)
    labels = model.labels_
    segment_2_sample_x['cluster{}'.format(nb_clusters)] = labels
segment_2_sample_x


,recency,frequency,montant,note_com,frais_livraison,temps_moyen_livraison,cluster
cluster,,,,,,,
-1,241.301038,3.412918,1666.301753,3.194642,49.157285,20.201845,867
0,236.873873,1.000000,141.072247,4.218297,19.552327,11.877050,23294
1,236.217376,2.000000,303.513640,3.902816,17.389682,10.804258,2912
2,238.627219,4.000000,508.882722,4.573964,15.221642,9.171598,169
3,236.136364,3.000000,473.509802,3.969697,16.220823,9.871542,506


In [ ]:
segment_2_sample_x.groupby('cluster5').agg({'recency' : 'mean',
                                'frequency' : 'mean',
                                'montant' : 'mean',
                                'note_com':'mean',
                                'frais_livraison':'mean',
                                'temps_moyen_livraison':'mean'
                                })

- Cluster 0 : Les clients de ce cluster sont des clients **"fidèles"** qui ont tendance à faire des achats fréquents et dépensent davantage sur chaque achat. Ils sont satisfaits de leurs achats et sont disposés à attendre un temps raisonnable pour la livraison. Cependant, ils peuvent être considérés comme des "clients inactifs" s'ils ne font pas d'achat pendant de longues périodes.

- Cluster 1 : Les clients de ce cluster sont des clients **"réguliers"** qui achètent moins souvent, mais ont tendance à donner des notes de satisfaction plus élevées pour leurs achats. Ils sont également moins dépensiers que ceux du Cluster 0, mais sont disposés à payer des frais de livraison raisonnables pour leurs achats. Ils peuvent également être considérés comme des "clients fidèles", car ils achètent régulièrement.

- Cluster 2 : Les clients de ce cluster ont une "période d'inactivité" élevée, une fréquence d'achat faible et des notes de satisfaction très basses. Ils sont moins dépensiers que ceux des clusters 0 et 1 et ont tendance à payer des frais de livraison plus élevés. Ils peuvent être considérés comme des **"clients en danger"**, car leur insatisfaction et leur faible fréquence d'achat pourraient conduire à une perte de clientèle.

- Cluster 3 : Les clients de ce cluster ont une "longue période d'inactivité", mais ont tendance à acheter avec une fréquence similaire à celle du cluster 1. Ils dépensent moins que ceux du cluster 0, mais ont des notes de satisfaction élevées pour leurs achats. Ils peuvent également être considérés comme des **"clients dormants"** qui pourraient être réactivés avec des incitations ou des promotions.

- Cluster 4 : Les clients de ce cluster ont une "période d'inactivité" similaire à celle du cluster 1, mais achètent légèrement moins souvent. Ils dépensent plus que ceux des autres clusters et ont des notes de satisfaction élevées pour leurs achats. Cependant, ils sont plus sensibles aux frais de livraison élevés et peuvent être considérés comme des **"clients à fidéliser"** qui pourraient être fidélisés avec des réductions de frais de livraison ou d'autres promotions avantageuses.

In [ ]:
segment_2_sample_x['cluster4'].value_counts()

In [ ]:
segment_2_sample_x.groupby('cluster4').agg({'recency' : 'mean',
                                'frequency' : 'mean',
                                'montant' : 'mean',
                                'note_com':'mean',
                                'frais_livraison':'mean',
                                'temps_moyen_livraison':'mean'
                                })

In [ ]:
segment_2_sample_x['profil client_cluster5'] = segment_2_sample_x['cluster5'].map({ 0: 'fidèles', 1:'réguliers', 2:'clients en danger', 3 :'clients dormants', 4: 'clients à fidéliser'})

#colors = cl.scales['9']['seq']['Blues']
colors = ['#FF0000', '#ff8000', '#00FF00', '##50C878']
fig = go.Figure( data = [go.Pie(values = segment_2_sample_x['profil client_cluster5'].value_counts().values,
                                labels = segment_2_sample_x['profil client_cluster5'].value_counts().index,
                                title = 'profil client_cluster5',
                                hole = .5,
                                textinfo='label+percent',
                                marker=dict(colors=colors),)
                                
                        ])
fig.update_layout(width=800, height=600)

fig.show()

In [ ]:
data=segment_2_sample_x.groupby('cluster5').agg({'recency' : 'mean',
                                'frequency' : 'mean',
                                'montant' : 'mean',
                                'note_com':'mean',
                                'frais_livraison':'mean',
                                'temps_moyen_livraison':'mean'
                                }).reset_index()
data

In [ ]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

# Charger les données
df = data

# Sélectionner les variables à visualiser
variables = ['recency', 'frequency', 'montant','note_com','frais_livraison','temps_moyen_livraison']

# Créer le scaler
scaler = MinMaxScaler()

# Appliquer le scaler sur les données
scaled_data = scaler.fit_transform(df[variables])

# Créer le dataframe avec les données scalées
df_scaled = pd.DataFrame(scaled_data, columns=variables)

# Ajouter la colonne cluster
df_scaled['cluster'] = df['cluster5'].values

# Créer un dataframe pour chaque cluster
clusters = df_scaled.groupby('cluster')

# Récupérer les noms des clusters
cluster_names = df_scaled['cluster'].unique()

# Créer la figure Plotly
fig = go.Figure()

# Parcourir les clusters et ajouter chaque radar plot à la figure
for i, cluster in enumerate(clusters):
    data = cluster[1][variables].mean().values.flatten().tolist()
    data += data[:1]
    fig.add_trace(go.Scatterpolar(
        r=data,
        theta=variables + [variables[0]],
        fill='toself',
        name=f'Cluster {cluster_names[i]}'
    ))

# Ajouter le titre et les étiquettes des axes
fig.update_layout(
    title='Radar plot des clusters',
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        ),
    ),
    showlegend=True,

    width=800,
    height=600
)

# Afficher la figure
fig.show()

# Projection sur le plan factoriel

In [ ]:
from sklearn import decomposition
pca=decomposition.PCA(n_components=2)
pca.fit (segment_2_scaled)
print( pca.explained_variance_ratio_.cumsum() )

In [ ]:
X_trans =pca.transform (segment_2_scaled)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

fig = plt.figure(figsize=(16,5))
# Agg Clustering with k=4
cls4 = AgglomerativeClustering(n_clusters=4)
cls4.fit(segment_2_scaled)

# Plot 1 - Agg Clustering (k=4)
plt.subplot(1, 2, 1)
sns.scatterplot(x=X_trans[:,0], y=X_trans[:,1], hue=cls4.labels_, palette='Set1')
sns.set_style("whitegrid")
plt.title('Agglomerative Clustering (k=4)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Agg Clustering with k=3
cls3 = AgglomerativeClustering(n_clusters=5)
cls3.fit(segment_2_scaled)

# Plot 2 - Agg Clustering (k=3)
plt.subplot(1, 2, 2)
sns.scatterplot(x=X_trans[:,0], y=X_trans[:,1], hue=cls3.labels_, palette='Set1')
sns.set_style("whitegrid")
plt.title('Agglomerative Clustering (k=5)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Adjust layout and show plot
plt.tight_layout()
plt.show()
